# 基础结构的ResNet block

In [11]:
'''
每个BasicBlock由两个卷积层组成，每个卷积层后面跟着一个批量归一化层和ReLU激活函数。
维度保持不变，即输入和输出的通道数相同。
'''
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1):
        super(ResidualBlock,self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,3,stride,1)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels,out_channels,3,1,1)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        #shortcut部分
        #默认是一个恒等映射
        self.shortcut=nn.Sequential()
        #如果输入与输出维度不一样，则需要调整
        if stride!=1 or in_channels!=out_channels:
            self.shortcut=nn.Sequential(
                nn.Conv2d(in_channels,out_channels,1,stride,0),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self,x):
        out=self.relu(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        out+=self.shortcut(x)
        out=self.relu(out)
        return out


x=torch.randn(64,3,224,224)
residual_block1=ResidualBlock(3,64)
out1=residual_block1(x)
residual_block2=ResidualBlock(3,64,2)
out2=residual_block2(x)
print(f"输入与输出是一个维度，恒等映射的情况下stride=1: {out1.shape}")
print(f"输入与输出维度不同，残差连接部分需要进行变化stride=2: {out2.shape}")
        


输入与输出是一个维度，恒等映射的情况下stride=1: torch.Size([64, 64, 224, 224])
输入与输出维度不同，残差连接部分需要进行变化stride=2: torch.Size([64, 64, 112, 112])


## 为了使网络加深，同时节约计算量，使用一个bottleneck结构

bottleneck的核心是先降维，再提取特征，再升维

In [9]:
class Bottleneck(nn.Module):
    '''
    瓶颈结构的ResNet block
    :param inplanes:输入通道数
    :param planes:输出通道数
    :param stride:步长
    :param downsample:下采样层
    downsample是为了处理当主分支的输出维度与残差分支的输出维度不同时，需要进行下采样的情况
    通常是用于判断残差流是用identity block还是conv block
    '''
    expansion=4#扩展后的维度的倍数,实际的输出通道数是planes*expansion
    def __init__(self,inplanes,planes,stride=1,downsample=None):
        super().__init__()
        self.conv1=nn.Conv2d(inplanes,planes,kernel_size=1,stride=1,bias=False)
        self.bn1=nn.BatchNorm2d(planes)

        self.conv2=nn.Conv2d(planes,planes,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(planes)

        self.conv3=nn.Conv2d(planes,planes*self.expansion,kernel_size=1,stride=1,bias=False)
        self.bn3=nn.BatchNorm2d(planes*self.expansion)

        self.relu=nn.ReLU(inplace=False)
        self.downsample=downsample
        self.stride=stride

    def forward(self,x):
        '''
        在这里实现残差块的连接，需要根据downsample是否为None来判断是否需要进行下采样
        如果是Identity Block，直接将输入x添加到输出上
        如果是Conv Block，需要先进行下采样，然后再添加到输出上
        identity是恒等映射，可以串联，增加网络深度
        conv是改变维度，无法继续串联
        '''
        residual=x# 残差连接的输入
        #print(f"residual.shape={residual.shape}")
        #print(f"x.shape={x.shape}")

        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        #print(f"conv1.shape={out.shape}")
        out=self.conv2(out)
        out=self.bn2(out)
        out=self.relu(out)
        #print(f"conv2.shape={out.shape}")
        out=self.conv3(out)
        out=self.bn3(out)
        #print(f"conv3.shape={out.shape}")

        if self.downsample is not None:
            #需要对残差块进行下采样
            residual=self.downsample(residual)
            #print(f"downsample.shape={residual.shape}")
        # 残差连接
        out+=residual
        out=self.relu(out)
        print(f"out.shape={out.shape}")
        return out


# 搭建完整的残差网络。

In [ ]:
class ResNet(nn.Module):
    def __init__(self,block,layers,num_classes=10):
        super().__init__()
        # -----------------------------------#
        #   假设输入进来的图片是3，224，224
        # -----------------------------------#
        self.inplanes=64
        '''
        ResNet共同的初始化操作都是：
        1.卷积层,使用大卷积核，减半尺寸
        2.批量归一化层
        3.激活函数层
        4.池化层，最大池化，继续减半尺寸
        目的是：迅速降低图像的分辨率，同时增加通道数
        '''
        self.conv1=nn.Conv2d(3,self.inplanes,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(self.inplanes)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        #定义残差块，只在第一个块中使用下采样
        self.layer1=self._make_layer(block,64,layers[0],stride=1)
    
        self.layer2=self._make_layer(block,128,layers[1],stride=2)
        self.layer3=self._make_layer(block,256,layers[2],stride=2)
        self.layer4=self._make_layer(block,512,layers[3],stride=2)

        #自适应平均池化层，将特征图的尺寸调整为1x1
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        #全连接层，将特征图的通道数映射到类别数
        self.fc=nn.Linear(512*block.expansion,num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        '''
        搭建一个残差块层
        block:残差块的类型
        planes*expansion:输出通道数
        blocks:残差块的数量
        stride:步长
        return :构造好的残差块层
        注意：在使用了 Bottleneck 的 ResNet中，
        每一层（Layer 1 到 Layer 4）的第一个 Block 都是 Conv Block，
        后面的才是 Identity Block。

        '''
        layers=[]
        #定义下采样块来处理输入和输出通道数不同的情况
        if stride!=1 or self.inplanes!=planes*block.expansion:
            downsample=nn.Sequential(
                nn.Conv2d(self.inplanes,planes*block.expansion,1,stride=stride,bias=False),
                nn.BatchNorm2d(planes*block.expansion)
            )
        else:
            downsample=None
        #第一层是Conv block，其余的是Identity block
        layers.append(block(self.inplanes,planes,stride,downsample))
        #更新通道数
        self.inplanes=planes*block.expansion
        #添加剩余的残差块
        for _ in range(1,blocks):
            layers.append(block(self.inplanes,planes))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        print(f"输入尺寸: {x.shape}")
        # 1. 卷积层, 使用大卷积核, 减半尺寸
        x=self.conv1(x)
        print(f"初始化卷积层输出尺寸: {x.shape}")
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        print(f"初始化池化层输出尺寸: {x.shape}")
        x=self.layer1(x)
        print(f"Layer1输出尺寸: {x.shape}")
        x=self.layer2(x)
        print(f"Layer2输出尺寸: {x.shape}") 
        x=self.layer3(x)
        print(f"Layer3输出尺寸: {x.shape}")
        x=self.layer4(x)    
        print(f"Layer4输出尺寸: {x.shape}")
        # 自适应平均池化层, 将特征图的尺寸调整为1x1
        x=self.avgpool(x)
        print(f"自适应平均池化层输出尺寸: {x.shape}")
        # 展平层, 将特征图展平为一维向量
        x=torch.flatten(x,1)#从第一个维度开始展开
        print(f"展平层输出尺寸: {x.shape}")
        # 全连接层, 将特征图的通道数映射到类别数
        x=self.fc(x)
        print(f"全连接层输出尺寸: {x.shape}")
        return x

X=torch.randn(1,3,224,224)# 批量大小为 3，通道数为 3，图像尺寸为 224x224
model=ResNet(Bottleneck,[2,2,2,2])
model(X) # Resnet18 18是指有权重的层数 比如卷积层和线性层 1+16+1=18

输入尺寸: torch.Size([1, 3, 224, 224])
初始化卷积层输出尺寸: torch.Size([1, 64, 112, 112])
初始化池化层输出尺寸: torch.Size([1, 64, 56, 56])
out.shape=torch.Size([1, 256, 56, 56])
out.shape=torch.Size([1, 256, 56, 56])
Layer1输出尺寸: torch.Size([1, 256, 56, 56])
out.shape=torch.Size([1, 512, 28, 28])
out.shape=torch.Size([1, 512, 28, 28])
Layer2输出尺寸: torch.Size([1, 512, 28, 28])
out.shape=torch.Size([1, 1024, 14, 14])
out.shape=torch.Size([1, 1024, 14, 14])
Layer3输出尺寸: torch.Size([1, 1024, 14, 14])
out.shape=torch.Size([1, 2048, 7, 7])
out.shape=torch.Size([1, 2048, 7, 7])
Layer4输出尺寸: torch.Size([1, 2048, 7, 7])
自适应平均池化层输出尺寸: torch.Size([1, 2048, 1, 1])
展平层输出尺寸: torch.Size([1, 2048])
全连接层输出尺寸: torch.Size([1, 10])


tensor([[ 0.0722,  0.1190, -0.1511,  0.9905,  0.3131,  0.4621,  0.0034, -0.7411,
          0.2812,  0.0759]], grad_fn=<AddmmBackward0>)